In [ ]:
!apt-get install -qq texlive texlive-latex-extra texlive-science

# Step 2: Python code for conversion
import subprocess
import os
from google.colab import files
# Load model directly
import re
from transformers import AutoTokenizer, AutoModelForCausalLM

Extracting templates from packages: 100%
Preconfiguring packages ...
Selecting previously unselected package fonts-droid-fallback.
(Reading database ... 124574 files and directories currently installed.)
Preparing to unpack .../00-fonts-droid-fallback_1%3a6.0.1r16-1.1build1_all.deb ...
Unpacking fonts-droid-fallback (1:6.0.1r16-1.1build1) ...
Selecting previously unselected package fonts-lato.
Preparing to unpack .../01-fonts-lato_2.0-2.1_all.deb ...
Unpacking fonts-lato (2.0-2.1) ...
Selecting previously unselected package poppler-data.
Preparing to unpack .../02-poppler-data_0.4.11-1_all.deb ...
Unpacking poppler-data (0.4.11-1) ...
Selecting previously unselected package tex-common.
Preparing to unpack .../03-tex-common_6.17_all.deb ...
Unpacking tex-common (6.17) ...
Selecting previously unselected package fonts-urw-base35.
Preparing to unpack .../04-fonts-urw-base35_20200910-1_all.deb ...
Unpacking fonts-urw-base35 (20200910-1) ...
Selecting previously unselected package libgs9-co

Currently choosing the R1-Distill-Qwen-1.5B
you can try other deepseek model.
if error happened when trying to load the model it usually caused by not enough of RAM

In [ ]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.06k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
prompt = """
generate me long article about algebra and calculus.
make it to be an article that contains more detail about the formula theory and more.
add author: Romadhon Wiratama in the article
"""

s_prompt = r"""
Role Definition: "You are an expert in Math Sciences assisting in creating structured, valid LaTeX documents for an article. The output must be free from syntax errors and ready for direct compilation."

Detailed Output Instructions:

Important Additional Details:
  - If the user specifies "author" in the prompt, include it as the author of the summarization report using the \author{} command in LaTeX.
  - If the user specifies "references" in the prompt, include it as the references section of the summarization report, using the \section*{References} environment.
  - If no "author" is specified, omit the \author{} command.
  - If no "references" are provided, omit the references section entirely.

Document Structure:
  - Every response must start with \documentclass{article}.
  - Include necessary packages for mathematical and scientific formatting, such as:
      [
        \usepackage{amsmath}
        \usepackage{amssymb}
        \usepackage{amsfonts}
      ]
  - Begin the document with \begin{document} and end it with \end{document}.
  - Ensure all content (title, sections, equations, etc.) is enclosed within the document environment.
  - Always validate that the class file (e.g., `article`) supports the `\normalsize` font size command to avoid errors like:
    ```
    ! LaTeX Error: The font size command \normalsize is not defined:
                   there is probably something wrong with the class file.
    ```
  - Use only standard and widely-supported document class options (e.g., `article`, `report`, `book`) to ensure compatibility.

Formatting Requirements:
  - Use proper LaTeX environments:
      > For lists: itemize or enumerate.
      > For equations: equation or align.
  - Avoid adding stray characters, unclosed environments, or any syntax that LaTeX cannot interpret.
  - If you are generating special characters (e.g., %, #, _), ensure they are escaped (e.g., \%, \#, \_).

Validation:
  - Always validate the document structure:
      > Check for matching \begin{} and \end{} pairs.
      > Ensure there is exactly one \begin{document} and one \end{document}.
  - Provide only the LaTeX content—do not include any plain text explanations or non-LaTeX code.

Filename:
  - Generate a short title as a filename, reflecting the topic (e.g., Quantum_Theory).
  - Exclude the filename from the document body.
"""

In [ ]:
# prompt = "Create a Python function to calculate Fibonacci sequence with O(n) time complexity"  #  user prompt
messages = [
    {"role": "system", "content": s_prompt},  #  system prompt
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

In [ ]:
# input processing logic
generated_ids = model.generate(**model_inputs, max_new_tokens=50000)
generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [ ]:
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
print(response)

<think>
Alright, so the user is asking me to generate an article about quantum physics theory. They want it to be a report with more detail and formulas. They also specified to include an author: Romadhon Wiratama. Hmm, okay.

First, I need to structure the LaTeX document properly. They mentioned starting with \documentclass{article}, so I'll set that up. They also want necessary packages, like amsmath, amsfonts, and amssymb. I'll include those in the packages section.

Next, the document should begin with \begin{document} and end with \end{document}. I'll make sure those are correctly placed. Since there's no mention of font size, I'll stick with \normalsize to avoid any issues.

They want the title, so I'll come up with something short but descriptive, like "Quantum Physics Theory: A Comprehensive Analysis." I'll place it inside the title option.

Now, the author section. They specified to include \author{}, so I'll add their name there. I'll make sure to format it correctly with the

In [ ]:
def process_response(response):
    # Remove <think> tags
    processed_text = re.sub(r'<think>.*?</think>', '', response, flags=re.DOTALL)
    processed_text = processed_text.strip()

    # Extract LaTeX content from code blocks
    code_blocks = re.findall(r'```(?:latex)?\s*(.*?)```', processed_text, flags=re.DOTALL)
    if code_blocks:
        processed_text = code_blocks[0].strip()
    else:
        processed_text = processed_text.strip()

    # Extract title from \title{}
    title_match = re.search(r'\\title\{(.*?)\}', processed_text, flags=re.DOTALL)
    title = title_match.group(1).strip() if title_match else "Untitled"

    # Capture FULL DOCUMENT STRUCTURE with preamble
    full_doc_match = re.search(
        r'(\\begin\{document\}.*?\\end\{document\})',
        processed_text,
        flags=re.DOTALL
    )

    if full_doc_match:
        body = full_doc_match.group(1).strip()
    else:
        # Fallback: Try to reconstruct minimal working document
        body = (
            "\\documentclass{article}\n"
            "\\usepackage{amsmath,amssymb}\n"
            "\\begin{document}\n"
            f"{processed_text}"
            "\n\\end{document}"
        )

    return processed_text, title, body

In [ ]:
p_response = list(process_response(response))
print(f'title: {p_response[1]}')
print(f'response: {p_response[0]}')
print(f'Body: {p_response[2]}')

title: Quantum Physics Theory: A Comprehensive Analysis
response: \documentclass{article}
\usepackage{amsmath}
\usepackage{amssymb}
\usepackage{amsfonts}

\title{Quantum Physics Theory: A Comprehensive Analysis}

\author{Romadhon Wiratama}

\begin{document}

\maketitle

\section{Introduction}
The study of quantum physics has revolutionized our understanding of the fundamental principles governing the behavior of matter and energy at the atomic and subatomic levels. This article aims to provide a comprehensive overview of quantum theory, exploring its foundational concepts, mathematical formulations, and applications in modern physics.

\section{Theoretical Framework}
The theoretical framework of quantum mechanics is built upon several key principles, including wave-particle duality, the uncertainty principle, and the principle of superposition. These principles form the basis for understanding quantum phenomena such as quantum entanglement and quantum tunneling.

\section{Mathematical 

In [ ]:
def tex_to_pdf_colab(tex_content, filename="output"):
  # Create temp directory
  os.makedirs("/content/temp", exist_ok=True)
  tex_path = f"/content/temp/{filename}.tex"

  # Write TeX file
  with open(tex_path, "w") as f:
    f.write(tex_content)

  # Compile using pdflatex
  try:
    result = subprocess.run(
        ["pdflatex", "-output-directory", "/content/temp", tex_path],
        capture_output=True,
        text=True
    )

    if result.returncode != 0:
      print("Error log:")
      print(result.stderr)
      return None
    else:
      print("PDF generated successfully!")
      return f"/content/temp/{filename}.pdf"

  except Exception as e:
    print(f"Error: {str(e)}")
    return None

In [ ]:
# Generate PDF
pdf_path = tex_to_pdf_colab(p_response[0],p_response[1])

# Download the PDF
if pdf_path and os.path.exists(pdf_path):
  files.download(pdf_path)
else:
  print("Failed to generate PDF")

PDF generated successfully!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>